<a href="https://colab.research.google.com/github/cccaaannn/machine_learning_colab/blob/master/house_prices/data_mining_hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


unzip dataset

In [ ]:
!unzip /content/drive/My\ Drive/datasets/house-prices-advanced-regression-techniques.zip

imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from scipy.stats import stats
from sklearn import tree
from IPython.display import Image 
import pydotplus

error functions

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def kaggle_score(y_true,y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
def print_errors(y_true, y_pred):
    print("mae:", mean_absolute_error(y_true, y_pred))
    print("mape:", mean_absolute_percentage_error(y_true, y_pred))
    print("kaggle-score:", kaggle_score(y_true, y_pred))


Load the data

In [3]:
train_data = pd.read_csv("train.csv", index_col=0)
test_data = pd.read_csv("test.csv", index_col=0)

Pearson correlation

In [ ]:
p_value_threshold = 0.05

num_cols = train_data._get_numeric_data().columns
pearson_corrs = []
for col in num_cols:
    s = stats.pearsonr(train_data.fillna(train_data.mean())[col], train_data['SalePrice'])
    if(s[1] < p_value_threshold):
        pearson_corrs.append((col, s[0], s[1]))

pearson_corrs = sorted(pearson_corrs, reverse=True, key=lambda tup: abs(tup[1]))

for i in pearson_corrs:
    print("columns: {0:<12} correlation: {1:<22} p-values: {2:<22}".format(i[0], i[1], i[2]))

select columns

In [ ]:
# x_columns = ["OverallQual", "GrLivArea", "GarageCars","GarageArea"]
x_columns = ["OverallQual", "GrLivArea", "GarageCars", "GarageArea", "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd", "MasVnrArea", "GarageYrBlt", "Fireplaces", "BsmtFinSF1"] 
y_column = 'SalePrice'
train_data[x_columns].head()

split train and validation

In [79]:
# x = train_data.loc[:,x_columns]
# x = train_data.drop(y_column, axis=1)
# x = pd.get_dummies(x)

x = train_data.loc[:, x_columns]
y = train_data.loc[:, y_column]

x_test = test_data.loc[:, x_columns]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

MinMaxScaler

In [80]:
x_train = x_train.fillna(x_train.mean())
x_val = x_val.fillna(x_val.mean())
x_test = x_test.fillna(x_test.mean())

scaler = MinMaxScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

LinearRegression

In [ ]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(x_train_scaled, y_train)
linear_regression_val_predictions = linear_regression_model.predict(x_val_scaled)

print_errors(y_val, linear_regression_val_predictions)

DecisionTreeRegressor

In [ ]:
tree_regression_model = DecisionTreeRegressor(max_depth=8)
tree_regression_model.fit(x_train_scaled, y_train)
tree_regression_val_predictions = tree_regression_model.predict(x_val_scaled)

print_errors(y_val, tree_regression_val_predictions)

show the tree

In [ ]:
dot_data = tree.export_graphviz(tree_regression_model, out_file=None) 
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

test predictions

In [91]:
test_predictions = linear_regression_model.predict(x_test_scaled)
linear_regression_test_predictions = pd.DataFrame({"Id" : test_data.index, "SalePrice" : test_predictions.astype(np.int32)})

test_predictions = tree_regression_model.predict(x_test_scaled)
tree_regression_test_predictions = pd.DataFrame({"Id" : test_data.index, "SalePrice" : test_predictions.astype(np.int32)})

save test predicitons

In [92]:
linear_regression_test_predictions.to_csv("linear_regression_test_predictions.csv", sep=',', index=False)
tree_regression_test_predictions.to_csv("tree_regression_test_predictions.csv", sep=',', index=False)